## Greenburgh Tax Assessment Analysis

### Setup

In [ ]:
## It is important that the tax_utils.py file (in src/utils) is accessible by your
## PYHTHONPATH environment variable.
## NOTE: If tax_utils.py is changed it will need to byte compiled before it can be used 
##       in this notebook.
##       To do this do the following:
##       cd src/utils
##       python -m py_compile tax_utils.py
import tax_utils as utils

## If you have support for jax, you can import of tax_utils with the following line.
#import tax_jax_utils as utils

import numpy as np
import pandas as pd
import matplotlib as plt
import sqlite3 as lite
import matplotlib
import matplotlib.pyplot as plt
plt.style.use('ggplot')
pd.set_option('display.max_columns', 15)

In [ ]:
## Tax assessment database and Assessment table.
TAX_ASSESS_DB = "/home/rsm/proj/tax_ass/taxdb/taxrec.db"
ASSESS_TABLE='taxrec'


In [ ]:
## Connect to database and read in data from table, <ASSESS_TABLE>.
conn  = lite.connect(TAX_ASSESS_DB)
query = f"SELECT * from {ASSESS_TABLE};"
df    = pd.read_sql_query(query, conn)

In [ ]:
## We need data sorted by year as we will group by YEAR to get a list of FULL_MKT_VALUE(s)
## which we will use to create a return series, and placed in a field in <df>.
df = df.sort_values(by='YEAR')

### Data Examination

In [ ]:
df.YEAR.unique()
df.shape

In [ ]:
df.head()

In [ ]:
df.PARCEL_TYPE.unique()

### Data Cleaning
We use regular expression matching to determine which data should be filtered out.
We create a dictionary below, badLinesDct, that contains the number of bad lines
for each field of interest: FULL_MKT_VALUE (market value of parcel), ACCT (parcel account id), LUC (Land Use Code), ACCR (parcel acreage size).

In [ ]:
badLinesDct = {}
good_mkt_filter = df["FULL_MKT_VALUE"].astype(str).str.match("^\d+$")
badLinesDct['FULL_MKT_VALUE']  = df.loc[~ good_mkt_filter].shape[0]

good_acct_filter = df["ACCT"].astype(str).str.match("^\d+$")
badLinesDct['ACCT'] = df.loc[~ good_acct_filter].shape[0]

good_luc_filter = df['LUC'].astype(str).str.match("^\d+$")
badLinesDct['LUC'] = df.loc[~ good_luc_filter].shape[0]

good_accr_filter = df['ACCR'].astype(str).str.match("(^(\d*)\.\d+$)|(^\d+(\.\d*)?$)")
badLinesDct['ACCR'] = df.loc[~ good_accr_filter].shape[0]

In [ ]:
## Filter out the bad mkt value, luc, and acct data.
df_filt = df[(good_mkt_filter & good_luc_filter & good_acct_filter)]

## The number of years of data.
NUMBER_OF_YEARS_OF_DATA=np.size(df_filt.YEAR.unique())

In [ ]:
print(f"Raw data shape             = {df.shape}")
print(f"After first cleaning shape = {df_filt.shape}")
print(f"Number of years of data    = {NUMBER_OF_YEARS_OF_DATA}")

In [ ]:
dd = df_filt.groupby('ACCT')['FULL_MKT_VALUE'].agg(lambda x: x.size).reset_index(name='MKT_COUNT')

dd1 = df_filt.groupby('ACCT')['FULL_MKT_VALUE'].agg(lambda x: any(x == 0)).reset_index(name='MKT_ZERO')

ddd = dd.merge(dd1, on='ACCT', how='inner')

## Now get the accounts that extend over the number of years of data we have that are "valid" in the sense that the value is non-zero.
accts = ddd.loc[(ddd.MKT_COUNT == NUMBER_OF_YEARS_OF_DATA) & (~ ddd.MKT_ZERO)].ACCT

## Only use these accounts from the filtered data. This is the data set we will use for analysis.
df_clean = df_filt.loc[df_filt['ACCT'].isin(accts), :]
df_clean.to_csv("clean_tax_ass.psv", sep='^', encoding='utf-8')

In [ ]:
print(f"Raw data shape              = {df.shape}")
print(f"After first cleaning shape  = {df_filt.shape}")
print(f"Number of accounts (should reduce size by a factor of {NUMBER_OF_YEARS_OF_DATA}) = {np.size(accts)}")


In [ ]:
## Describe the reduction in data after cleaning.
print(f"Data cleaning reduced the overall data set by {100.0 * np.round( (df.shape[0] - df_clean.shape[0]) / df.shape[0], 2)}%.")
raw_residencial_count = df.loc[df.LUC == 210].shape[0]
filtered_residencial_count = df_clean.loc[df_clean.LUC == 210].shape[0]
print(f"Data cleaning reduced the residencial data set by {100.0 * np.round( (raw_residencial_count - filtered_residencial_count) / raw_residencial_count, 2)}%.")

### Compute Market Return Series

In [ ]:
## Create a field, 'mkt_vals' that is an np.array of returns (ordered by YEAR).
print(f"Shape of df_clean = {df_clean.shape}")
df_rets = df_clean.groupby('ACCT').apply(lambda row: np.array(row['FULL_MKT_VALUE'])).reset_index(name='mkt_vals')
print(f"Shape of df_rets = {df_rets.shape}")
df_rets['mkt_rets'] = df_rets['mkt_vals'].apply(lambda x: np.diff(x)) / df_rets['mkt_vals'].apply(lambda x: x[:-1])

df_rets['avg_mkt_val'] = df_rets.apply(lambda row: np.mean(row['mkt_vals']), axis=1)

df_rets = df_clean.merge(df_rets, on='ACCT', how='inner')
print(f"Shape of df_rets = {df_rets.shape}")

## Pick the first year of the data, we have all of the return and market values for all years stored as vectors.
## So, the fields for most things don't change over the years; however, OWN1 and OWN2 can most likely change.
df_rets = df_rets.loc[df_rets.YEAR == df_rets.YEAR.unique().min(), :]

In [ ]:
df_rets.columns

### Examine Parcel-Type Counts

In [ ]:
pv = pd.pivot_table(df_rets[['LUC', 'PARCEL_TYPE']], index=['PARCEL_TYPE', 'LUC'], aggfunc=lambda x: int(x.size / 2)).to_frame(name="LUC_cnt").reset_index()
pv['LUC_log10_cnt'] = np.log10(pv['LUC_cnt'])
ax = pv.plot.scatter(x = 'LUC', y='LUC_log10_cnt', xlabel='LUC\n(Residencial LUC=210)', ylabel='Log10 of LUC Count', title="Log10 of LUC counts")
ax.axvline(210, linestyle='--');


In [ ]:
## The top 10 Parcel-Types by count along with their description.
pv.sort_values(by='LUC_cnt', ascending=False).head(10)

### Aggregate Market Returns and Cumulative Returns

In [ ]:
## Empty data frame to store returns.
df_results = pd.DataFrame({})

In [ ]:
## Overall Aggregated Market Returns
df_results['overall'] = df_rets['mkt_rets'].agg(np.mean)

In [ ]:
## Aggregated Market Returns for LUC 210 -- Single family residence.
df_results['residence'] = df_rets.loc[df_rets['LUC'] == 210, 'mkt_rets'].agg(np.mean)

In [ ]:
## Compute overall weighted returns using the average market value as the weight 
## -- Also recompute but restrict analysis to single family residences -- LUC = 210.
df_results['overall_mkt_wgt']       = utils.assessment_agr_rets(df_rets, 'mkt_rets', 'avg_mkt_val')
df_results['residence_mkt_wgt']     = utils.assessment_agr_rets(df_rets, 'mkt_rets', 'avg_mkt_val', filt = df_rets['LUC'] == 210)
df_results['overall_mkt_wgt_med']   = utils.assessment_wgt_quant_rets(df_rets, 'mkt_rets', 'avg_mkt_val', np.array([0.5]))[:, 0] 
df_results['residence_mkt_wgt_med'] = utils.assessment_wgt_quant_rets(df_rets, 'mkt_rets', 'avg_mkt_val', np.array([0.5]), filt=df_rets['LUC'] == 210)[:, 0]


In [ ]:
ax = df_results[['residence', 'overall_mkt_wgt', 'overall_mkt_wgt_med', 'residence_mkt_wgt', 'residence_mkt_wgt_med']].plot( 
                     xlabel="Year"                     , 
                     ylabel="Assessment Change from Previous Year",
                     title="Tax Assessment Yearly Change Comparison (Greenburgh)" ,
                     xticks=df_results.index, rot=90    )
ax.set_xticklabels(['2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']);
ax.legend(['Residential Avg', 'Overall Mkt Wgt Avg', 'Overall Mkt Wgt Med', 'Residential Mkt Wgt Avg', 'Residential Mkt Wgt Med']);

In [ ]:
## Compute various cumulative weighted returns.

## Overall Aggregated (Avg) Market cumulative Returns
df_results['overall_cs'] = np.cumprod(1.0 + df_rets['mkt_rets'].agg(np.mean)) - 1.0

## Residential Aggregated (Avg) Market Cumulative Returns for LUC 210 -- Single family residence.
df_results['residence_cs'] = np.cumprod(1.0 + df_rets.loc[df_rets['LUC'] == 210, 'mkt_rets'].agg(np.mean)) - 1.0

## Repeat the above with Aggregated (Avg and Med) weighted returns using the average market value as the weight.
df_results['overall_mkt_wgt_cs'] = np.cumprod(1.0 + utils.assessment_agr_rets(df_rets, 'mkt_rets', 'avg_mkt_val')) - 1.0
df_results['overall_mkt_med_cs'] = np.cumprod(1.0 + utils.assessment_wgt_quant_rets(df_rets, 'mkt_rets', 'avg_mkt_val', np.array([0.5]))[:, 0]) - 1.0 

In [ ]:
## Compute overall weighted returns using the average market value as the weight 
## -- but restrict analysis to single family residences -- LUC = 210.
df_results['residence_mkt_wgt_cs'] = np.cumprod(1.0 + utils.assessment_agr_rets(df_rets, 'mkt_rets', 'avg_mkt_val', filt = df_rets['LUC'] == 210)) - 1.0
df_results['residence_mkt_med_cs'] = np.cumprod(1.0 + utils.assessment_wgt_quant_rets(df_rets, 'mkt_rets', 'avg_mkt_val', np.array([0.5]), filt=df_rets['LUC'] == 210)[:, 0]) - 1.0 

In [ ]:
ax = df_results[['residence_cs', 'overall_mkt_wgt_cs', 'overall_mkt_med_cs', 'residence_mkt_wgt_cs', 'residence_mkt_med_cs']].plot( 
                     xlabel="Year"                     , 
                     ylabel="Assessment Change from 2012",
                     title="Tax Assessment Comparison (Greenburgh)\n(Cumulative Change)" ,
                     xticks=df_results.index, rot=90    )
ax.set_xticklabels(['2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']);
ax.legend(['Residential Avg', 'Overall Mkt Wgt Avg', 'Overall Mkt Med', 'Residential Mkt Wgt Avg', 'Residential Mkt Med']);